In [1]:
# 1. 请先使用上位机升级机器人电机通信固件版本到0.3.12.12，详细使用请参考上位机使用说明
# 2. 请使用上位机对机器人零位进行标定
# 3. 请使用上位机调整机器人电机pd参数（注意不是pid参数），参考值：上肢（头腰双臂）p：200，d：0，可以根据软硬自己调节
# 4. 请使用指定的docker镜像进行开发，镜像名为fourier_hardware:v2.2，请使用开源仓库中的docker启动脚本启动docker

In [2]:
# fourier_hardware_py 用于对机器人进行分组控制
import fourier_hardware_py
# grx_sot_py 用于对机器人进行运动学，动力学求解
import grx_sot_py

In [3]:
# 加载对应机型的config配置文件
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr2t2")

In [4]:
# 配置各控制组的控制模式为PD模式，并忽略下肢控制的运动
config.changeControlGroupMotorMode("head","pd")
config.changeControlGroupMotorMode("left_manipulator","pd")
config.changeControlGroupMotorMode("right_manipulator","pd")
config.changeControlGroupMotorMode("waist","pd")
config.changeControlGroupMotorMode("left_leg","ignore")
config.changeControlGroupMotorMode("right_leg","ignore")

True

In [5]:
# 此处会尝试与机器人建立连接，如果失败（存在control group无法连接），将会持续报错，请先检查机器人关节ip是否可以ping通，
# 如果不能ping通，请尝试重启机器人或寻求硬件支持
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/gr2t2",config)

In [6]:
# 根据配置文件定义对应的机器人类，支持运动学、动力学求解
robot = grx_sot_py.RobotWrapper("/usr/local/config/fourier_hardware/gr2t2", grx_sot_py.Flags.ignore_collisions)

In [7]:
# 定义机器人对应的控制组和与控制组匹配的关节顺序，输出对应的字典信息
group_names = []
group_joints = []
group_joints_pos_order = {}
group_joint_vel_order = {}

for group_name in config.getControlGroupNames():
    if config.getControlGroupMotorMode(group_name)== "ignore":
        continue
    group_names.append(group_name)
    robot_joint_names = hardware.getControlGroupJointNames(group_name)
    group_joints.append(robot_joint_names)
    joint_pos_orders = []
    joint_vel_orders = []
    for robot_joint_name in robot_joint_names:
        joint_pos_orders.append(robot.get_joint_offset(robot_joint_name))
        joint_vel_orders.append(robot.get_joint_v_offset(robot_joint_name))
    group_joints_pos_order[group_name]= joint_pos_orders
    group_joint_vel_order[group_name] = joint_vel_orders

In [8]:
print(config.getRobotName())
print(config.getControlGroupNames())

gr2t2
['waist', 'head', 'left_manipulator', 'right_manipulator', 'left_leg', 'right_leg']


In [9]:
# 使能整个机器人
hardware.enableRobot()

MOTOR: 192.168.137.93, ENABLE SUCCESS! 
MOTOR: 192.168.137.95, ENABLE SUCCESS! 
MOTOR: 192.168.137.10, ENABLE SUCCESS! 
MOTOR: 192.168.137.11, ENABLE SUCCESS! 
MOTOR: 192.168.137.12, ENABLE SUCCESS! 
MOTOR: 192.168.137.13, ENABLE SUCCESS! 
MOTOR: 192.168.137.14, ENABLE SUCCESS! 
MOTOR: 192.168.137.15, ENABLE SUCCESS! 
MOTOR: 192.168.137.16, ENABLE SUCCESS! 
MOTOR: 192.168.137.30, ENABLE SUCCESS! 
MOTOR: 192.168.137.31, ENABLE SUCCESS! 
MOTOR: 192.168.137.32, ENABLE SUCCESS! 
MOTOR: 192.168.137.33, ENABLE SUCCESS! 
MOTOR: 192.168.137.34, ENABLE SUCCESS! 
MOTOR: 192.168.137.35, ENABLE SUCCESS! 
MOTOR: 192.168.137.36, ENABLE SUCCESS! 
MOTOR: 192.168.137.90, ENABLE SUCCESS! 


True

In [10]:
# 获取机器人当前状态下的对应控制组['waist', 'head', 'left_manipulator', 'right_manipulator', 'left_leg', 'right_leg']的关节qpos(rad),
lm = hardware.getControlGroupState("left_manipulator")
rm = hardware.getControlGroupState("right_manipulator")
print("left_manipulator:", lm.q)
print("right_manipulator", rm.q)

left_manipulator: [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.52573731e-01
 -4.98777240e-03  1.83039341e+00 -3.08949024e-01  3.32594166e-01
 -3.84696547e-03 -1.73836642e-04]
right_manipulator [ 0.          0.          0.          0.          0.          0.
  0.         -0.36235867 -0.07474273 -0.26627685 -0.89162508  0.0326417
 -0.00601916 -0.00322142]


In [11]:
# 生成机器人双臂关节从当前位置到零位的轨迹
import numpy as np
point_init = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
rm_traj = fourier_hardware_py.bridgeTrajectory(rm.q[-7:],point_init)
lm_traj = fourier_hardware_py.bridgeTrajectory(lm.q[-7:],point_init)

[2025-02-28 09:54:10] [12182] [error] head loss message more than 50 times .
[2025-02-28 09:54:10] [12182] [error] left_manipulator loss message more than 50 times .
[2025-02-28 09:54:10] [12182] [error] right_manipulator loss message more than 50 times .


In [12]:
# 执行回零轨迹，驱动右臂控制组关节回到零位
import time

for i in range(len(rm_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
    params = robot.get_inertia_params()
    gravity_torque = regressor.dot(params)
    hardware.setControlGroupPosCmd("right_manipulator",rm_traj[1][i],np.zeros(7),gravity_torque[group_joint_vel_order["right_manipulator"]])
    state = hardware.readRealRobotState()

    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

control loop is slow 0.0026811319985426962
control loop is slow 0.0025902639972628094
control loop is slow 0.002563574002124369
control loop is slow 0.003718835003382992
control loop is slow 0.0034670360037125647
control loop is slow 0.005823382998642046
control loop is slow 0.0032039040015661158
control loop is slow 0.004924918997858185
control loop is slow 0.00426234099722933
control loop is slow 0.0035287989958305843
control loop is slow 0.003264264996687416
control loop is slow 0.004249937002896331
control loop is slow 0.004171569999016356
control loop is slow 0.005014800000935793
control loop is slow 0.0026811830030055717
control loop is slow 0.005989329998556059
control loop is slow 0.00377763500000583
control loop is slow 0.002846271003363654
control loop is slow 0.004583775997161865
control loop is slow 0.0034601810039021075
[2025-02-28 09:54:11] [12182] [error] head loss message more than 50 times .
[2025-02-28 09:54:11] [12182] [error] left_manipulator loss message more than 

In [13]:
# 驱动左臂控制组关节回到零位
import time

for i in range(len(lm_traj[1])):
    start_time = time.perf_counter()
    # 获取机器人真机的状态信息
    
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
    params = robot.get_inertia_params()
    gravity_torque = regressor.dot(params)
    hardware.setControlGroupPosCmd("left_manipulator",lm_traj[1][i],np.zeros(7),gravity_torque[group_joint_vel_order["left_manipulator"]])
    state = hardware.readRealRobotState()
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)

control loop is slow 0.0035979499953100458
control loop is slow 0.0031829810031922534
control loop is slow 0.002951706999738235
control loop is slow 0.0027888730037375353
control loop is slow 0.002732542998273857
control loop is slow 0.0031189979999908246
control loop is slow 0.003652622996014543
control loop is slow 0.005430723002064042
control loop is slow 0.004012568999314681
control loop is slow 0.003753549994144123
control loop is slow 0.002972766997118015
control loop is slow 0.0038476890040328726
control loop is slow 0.004045516994665377
control loop is slow 0.004920083003526088
control loop is slow 0.004913295000733342
control loop is slow 0.004917271995509509
control loop is slow 0.002784468000754714
control loop is slow 0.003049658000236377
control loop is slow 0.004205664998153225
control loop is slow 0.0030162700059008785
control loop is slow 0.0032116400034283288
control loop is slow 0.008058083003561478
control loop is slow 0.0038630829949397594
control loop is slow 0.006

In [14]:
# 从模型计算全身关节对重力进行补偿的力矩值
regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
params = robot.get_inertia_params()
gravity_torque = regressor.dot(params)
print(gravity_torque)
print(gravity_torque[group_joint_vel_order["left_manipulator"]])

[-8.24266348e-06 -1.65008281e-04  6.07729500e+02  1.25488915e-01
  4.00350906e+00  6.03871931e-04  5.89067276e-01 -3.75361564e-01
  0.00000000e+00 -5.94778606e-01  7.34956960e-03 -3.90931443e-01
  5.88970731e-01  3.75834211e-01  0.00000000e+00 -5.94731910e-01
 -6.98075185e-03 -3.90869640e-01  6.03871931e-04  5.01668601e-06
 -4.76477276e-01 -2.53120395e-01  6.49996393e-02 -7.98566600e-04
 -7.34682991e-02 -3.78607220e-04 -4.54717578e-02 -8.62739003e-04
 -2.60029229e-01  3.90202240e-02  7.52202066e-04 -8.34847611e-02
  7.65621794e-04 -4.78804784e-02  2.86750959e-03]
[-0.2531204   0.06499964 -0.00079857 -0.0734683  -0.00037861 -0.04547176
 -0.00086274]
[2025-02-28 09:32:30] [11127] [error] left_manipulator loss message more than 50 times .
[2025-02-28 09:32:30] [11127] [error] right_manipulator loss message more than 50 times .
[2025-02-28 09:32:31] [11127] [error] left_manipulator loss message more than 50 times .
[2025-02-28 09:32:31] [11127] [error] right_manipulator loss message more t

In [ ]:
import numpy as np
import math
position_desired = np.zeros(17)  # deg
ERROR_CODE = 50 #关节组报错的错误码

# 设置循环频率的时间
sim_dt = 0.0025
sim_duration = 60.00
sim_steps = int(sim_duration / sim_dt)

for k in range(sim_steps):
    start_time = time.perf_counter()
    # current control
    position_desired[0] = (15 * math.sin(0.0025 * k - math.pi / 2) + 15) # waist_yaw_joint

    position_desired[1] = -(15 * math.sin(0.0025 * k)) # head_yaw_joint
    position_desired[2] = -(15 * math.sin(0.0025 * k)) # head_pitch_joint
    
    position_desired[3] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_pitch_joint
    position_desired[4] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # left_shoulder_roll_joint
    position_desired[5] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_shoulder_yaw_joint
    position_desired[6] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_elbow_pitch_joint
    position_desired[7] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # left_wrist_yaw_joint
    # position_desired[8] = -(10 * math.sin(0.0025 * k)) # left_wrist_roll_joint
    # position_desired[9] = -(10 * math.sin(0.0025 * k)) # left_wrist_pitch_joint
    
    position_desired[10] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_pitch_joint
    position_desired[11] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_shoulder_roll_joint
    position_desired[12] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_shoulder_yaw_joint
    position_desired[13] = -(40 * math.sin(0.0025 * k - math.pi / 2) + 40) # right_elbow_pitch_joint
    position_desired[14] = 40 * math.sin(0.0025 * k - math.pi / 2) + 40 # right_wrist_yaw_joint
    # position_desired[15] = -(10 * math.sin(0.0025 * k)) # right_wrist_roll_joint
    # position_desired[16] =  -(10 * math.sin(0.0025 * k)) # right_wrist_pitch_joint

    #将控制命令从关节角度转换为弧度
    position_desired = np.deg2rad(position_desired)

    # 返回一个字典，字典的key是control group 的名字，value是连续失去连接的次数（最大观测值是50），如果value等于50，代表该control group 失去连接
    # 应该在程序控制周期内实时观测次值，保证程序运行的安全
    motor_stats = hardware.getGroupsLoss()
    #检查机器人真机的硬件状态，如果机器人使能后某个电机断连，将会切断控制循环命令，并答应错误信息。
    if ERROR_CODE in motor_stats.values():
        print(f"motor get error.")
        break

    # 获取机器人真机的状态信息
    state = hardware.readRealRobotState()
    # 由于机器人关节速度和关节加速度观测存在误差，且后续矩阵相乘带来延迟，所以这里不考虑科氏力和惯性力，只考虑重力（对state.qd和state.qdd置零）
    state.qd = np.zeros((state.qd).size)
    state.qdd = np.zeros((state.qdd).size)
    # 计算机器人的关节重力补偿力矩，并通过PD的前馈补偿该部分力矩，提高控制精度
    regressor = robot.get_joint_torque_regressor(state.q,state.qd,state.qdd)
    params = robot.get_inertia_params()
    gravity_torque = regressor.dot(params)
    
    # 针对双臂控制组进行控制，需要注意对应的输入：关节控制组、对应维度的关节位置信息、速度信息(设置为0)、前馈的力矩值
    # hardware.setControlGroupPosCmd("left_manipulator",position_desired[3:10],np.zeros(7),gravity_torque[group_joint_vel_order["left_manipulator"]])
    hardware.setControlGroupPosCmd("right_manipulator",position_desired[10:17],np.zeros(7),gravity_torque[group_joint_vel_order["right_manipulator"]])
    # hardware.setControlGroupPosCmd("waist", np.array([position_desired[0]]))
    # hardware.setControlGroupPosCmd("head", position_desired[1:3])
    
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/400 :
        time.sleep(0.0025-execution_time)
    else:
        print("control loop is slow",execution_time)


control loop is slow 0.0025294090009992942
control loop is slow 0.0026095930006704293
control loop is slow 0.003822819999186322
control loop is slow 0.004449412997928448
control loop is slow 0.0057928550013457425
control loop is slow 0.003243437997298315
control loop is slow 0.003100759000517428
control loop is slow 0.0030445750016951934
control loop is slow 0.005200824001803994
control loop is slow 0.002908579001086764
control loop is slow 0.0034603439999045804
control loop is slow 0.0028293900031712838
control loop is slow 0.003219023004930932
control loop is slow 0.0036417529991013
control loop is slow 0.002816312000504695
control loop is slow 0.004178545001195744
control loop is slow 0.004736720002256334
control loop is slow 0.003502065999782644
control loop is slow 0.003769399998418521
control loop is slow 0.007486918999347836
control loop is slow 0.0028827810019720346
control loop is slow 0.004826846998184919
control loop is slow 0.0030615729992859997
control loop is slow 0.00349

In [14]:
hardware.disableRobot()

MOTOR: 192.168.137.93, DISABLE SUCCESS! 


True

MOTOR: 192.168.137.95, DISABLE SUCCESS! 
MOTOR: 192.168.137.10, DISABLE SUCCESS! 
MOTOR: 192.168.137.11, DISABLE SUCCESS! 
MOTOR: 192.168.137.12, DISABLE SUCCESS! 
MOTOR: 192.168.137.13, DISABLE SUCCESS! 
MOTOR: 192.168.137.14, DISABLE SUCCESS! 
MOTOR: 192.168.137.15, DISABLE SUCCESS! 
MOTOR: 192.168.137.16, DISABLE SUCCESS! 
MOTOR: 192.168.137.30, DISABLE SUCCESS! 
MOTOR: 192.168.137.31, DISABLE SUCCESS! 
MOTOR: 192.168.137.32, DISABLE SUCCESS! 
MOTOR: 192.168.137.33, DISABLE SUCCESS! 
MOTOR: 192.168.137.34, DISABLE SUCCESS! 
MOTOR: 192.168.137.35, DISABLE SUCCESS! 
MOTOR: 192.168.137.36, DISABLE SUCCESS! 
MOTOR: 192.168.137.90, DISABLE SUCCESS! 
[2025-02-28 09:55:32] [12182] [error] head loss message more than 50 times .
[2025-02-28 09:55:32] [12182] [error] left_manipulator loss message more than 50 times .
[2025-02-28 09:55:32] [12182] [error] right_manipulator loss message more than 50 times .
[2025-02-28 09:55:33] [12182] [error] head loss message more than 50 times .
[2025-02-28